In [1]:
!pip install datasets scikit-learn python-dotenv


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from huggingface_hub import login
from dotenv import dotenv_values

login(token=dotenv_values('.env')['HUGGING_FACE_TOKEN'])

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\hakatashi\.cache\huggingface\token
Login successful


In [3]:
from datasets import load_dataset

dataset = load_dataset("hakatashi/hakatashi-pixiv-bookmark-deepdanbooru-private")

In [4]:
dataset

DatasetDict({
    test: Dataset({
        features: ['key', 'tag_probs', 'class'],
        num_rows: 59707
    })
    train: Dataset({
        features: ['key', 'tag_probs', 'class'],
        num_rows: 179121
    })
    validation: Dataset({
        features: ['key', 'tag_probs', 'class'],
        num_rows: 59708
    })
})

In [5]:
dataset['test'].features

{'key': Value(dtype='string', id=None),
 'tag_probs': Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None),
 'class': ClassLabel(names=['not_bookmarked', 'bookmarked_public', 'bookmarked_private'], id=None)}

In [6]:
np_dataset = dataset.with_format(type='numpy')

In [7]:
train_data = np_dataset['train'][:]
test_data = np_dataset['test'][:]
validation_data = np_dataset['validation'][:]

In [8]:
from sklearn import svm
svc = svm.LinearSVC(verbose=1)
svc.max_iter = 1000
svc.fit(train_data['tag_probs'], train_data['class'])

C:\Users\hakatashi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


[LibLinear]

LinearSVC(verbose=1)

In [9]:
svc.score(test_data['tag_probs'], test_data['class'])

0.8756092250489892

In [14]:
from joblib import load
svc = load('sklearn-multiclass-linear-svc.joblib')
from datasets import load_dataset
dataset = load_dataset("hakatashi/hakatashi-pixiv-bookmark-deepdanbooru-private")
np_dataset = dataset.with_format(type='numpy')
test_data = np_dataset['test'][:]
validation_data = np_dataset['validation'][:]

In [15]:
predicted_class = svc.predict(test_data['tag_probs'])

In [16]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
print('confusion_matrix:')
print(confusion_matrix(test_data["class"], predicted_class))
print(f'accuracy_score: {accuracy_score(test_data["class"], predicted_class)}')
print(f'precision_score: {precision_score(test_data["class"], predicted_class, average="macro")}')
print(f'recall_score: {recall_score(test_data["class"], predicted_class, average="macro")}')
print(f'f1_score: {f1_score(test_data["class"], predicted_class, average="macro")}')

confusion_matrix:
[[38220  2071   407]
 [ 2623 11715   181]
 [ 1348   797  2345]]
accuracy_score: 0.8756092250489892
precision_score: 0.8362452405559319
recall_score: 0.7560859845526466
f1_score: 0.7863721599938748


In [26]:
test_data_cat0 = list(filter(lambda res: res[1] == 0, zip(svc.decision_function(test_data['tag_probs']), test_data['class'], test_data['key'])))
validation_data_cat0 = list(filter(lambda res: res[1] == 0, zip(svc.decision_function(validation_data['tag_probs']), validation_data['class'], validation_data['key'])))
print('\n'.join(list(map(lambda res: res[2], sorted(test_data_cat0 + validation_data_cat0, key=lambda res: -res[0][2])[:100]))))

danbooru/5744585.jpg
danbooru/3822756.jpg
danbooru/1381355.jpg
danbooru/1195304.jpg
danbooru/4108633.jpg
danbooru/5328907.jpg
danbooru/1381369.jpg
danbooru/4030194.jpg
danbooru/1267625.jpg
danbooru/4280681.jpg
danbooru/2925207.jpg
danbooru/4732911.jpg
danbooru/4280678.jpg
danbooru/4266904.jpg
danbooru/4520274.jpg
danbooru/5329549.jpg
danbooru/5615702.jpg
danbooru/3718886.jpg
danbooru/5484267.jpg
danbooru/3314977.jpg
danbooru/4268406.jpg
danbooru/5576409.jpg
danbooru/3981263.jpg
danbooru/3861207.jpg
danbooru/5030083.jpg
danbooru/1747362.jpg
danbooru/1957957.jpg
danbooru/3755506.jpg
danbooru/5957548.jpg
danbooru/2786053.jpg
danbooru/5743517.jpg
danbooru/3859159.jpg
danbooru/5128402.jpg
danbooru/1381282.jpg
danbooru/3489479.jpg
danbooru/3314978.jpg
danbooru/5030074.jpg
danbooru/2793645.jpg
danbooru/5331042.jpg
danbooru/1260793.jpg
danbooru/2793647.jpg
danbooru/3998739.jpg
danbooru/5538230.jpg
danbooru/3315861.jpg
danbooru/4110517.jpg
danbooru/4276929.jpg
danbooru/4170131.jpg
danbooru/2924

In [25]:
len(list(filter(lambda tup: tup[0] == 2 and tup[1] != 2, zip(predicted_class, test_data['class'], test_data['key']))))

588

In [58]:
from joblib import dump
dump(svc, 'sklearn-multiclass-linear-svc.joblib')

['sklearn-multiclass-svc.joblib']